# Circuit simulation campaign setup for tiny circuits

In [ ]:
import obi_one as obi
from pathlib import Path

In [ ]:
circuit_path_prefix = Path("../../examples/data/tiny_circuits")

### Loading a circuit

In [ ]:
circuit = obi.Circuit(name="N_10__top_nodes_dim6", path=str(circuit_path_prefix / "N_10__top_nodes_dim6" / "circuit_config.json"))
nodes = circuit.sonata_circuit.nodes["S1nonbarrel_neurons"]
edges = circuit.sonata_circuit.edges["S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"]
print(f"Circuit '{circuit}' with {nodes.size} neurons and {edges.size} synapses")
display(nodes.get())

### Defining the circuit simulation

In [ ]:
# Simulation init
sim_neuron_set = obi.PredefinedNeuronSet(node_set="All")
sim_duration = 3000.0  # ms
simulations_initialize = obi.CircuitSimulationScanConfig.Initialize(
    circuit=circuit,
    node_set=sim_neuron_set,
    simulation_length=sim_duration,
)

# Stimuli
stim_neuron_set = obi.PredefinedNeuronSet(node_set="All", sample_percentage=[50, 100])
stim_times = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=1000.0)  # in ms!!
current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=stim_times, duration=5.0, neuron_set=stim_neuron_set, amplitude=[0.2, 0.5])

# Recordings
rec_neuron_set = obi.PredefinedNeuronSet(node_set="All")
v_recording = obi.SomaVoltageRecording(start_time=0.0, end_time=sim_duration, neuron_set=rec_neuron_set)

"""
Fill form with Blocks
"""
simulations_form = obi.CircuitSimulationScanConfig(
                                    initialize=simulations_initialize,
                                    intracellular_location_sets={},
                                    extracellular_locations={},
                                    neuron_sets={"SimNrnSet": sim_neuron_set, "StimNrnSet": stim_neuron_set, "RecNrnSet": rec_neuron_set},
                                    synapse_sets={},
                                    timestamps={"StimTimes": stim_times},
                                    stimuli={"IStim": current_stimulus},
                                    recordings={"SomaVoltRec": v_recording},
                                    )

In [ ]:
simulations_form.model_dump(mode="json")

In [ ]:
grid_scan = obi.GridScanGenerationTask(form=simulations_form, output_root='../../../obi-output/tiny_circuit_simulation_campaign/grid_scan')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)
grid_scan.execute(processing_method='generate')

In [ ]:
# Deserialization
# grid_scan_ds = obi.deserialize_obi_object_from_json_file("../../../obi-output/tiny_circuit_simulations/grid_scan/obi_one_scan.json")